In [ ]:
import pandas as pd
import shap
from sklearn.ensemble import RandomForestClassifier
from pycaret.classification import *
import numpy as np
from sklearn.model_selection import cross_val_score, RepeatedKFold, StratifiedKFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from tsfresh import extract_features, select_features
import matplotlib.pyplot as plt
from umap import UMAP
import umap.plot
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import accuracy_score, silhouette_score
from sklearn.preprocessing import label_binarize
from scipy.stats import chi2_contingency, shapiro, f_oneway, norm, ttest_ind
import warnings
from sklearn.linear_model import LinearRegression
import seaborn as sns
from matplotlib.colors import LogNorm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
prs_matrix = pd.read_csv('FinalData/all_CG_PRS_SNPs.raw', sep=' ') # prs snps (0,1,2)
prs_prof = pd.read_csv('../PRS_data/Cooccur_prs/autism_CG_PRS_only.profile', delim_whitespace=True) # actual prs scores
prs_prof['ZSCORE'] = (prs_prof['SCORE']-np.mean(prs_prof['SCORE']))/np.std(prs_prof['SCORE'])

In [ ]:
# prepare the prs matrix for random forest, same as done for prediction but this time use whole matrix (i.e. no training split)
prs_CG = prs_matrix.drop(columns=['FID', 'PAT', 'MAT', 'PHENOTYPE', 'SEX'])
prs_CG = prs_CG.set_index(['IID'])

pheno_CG = prs_matrix['PHENOTYPE']
sex_CG = prs_matrix['SEX']
val = []
for i in range(len(pheno_CG)):
    if pheno_CG[i] != 1 and pheno_CG[i] != 2:
        val.append(i)
pheno_CG = pheno_CG.drop(val)
sex_CG = sex_CG.drop(val)
prs_prof = prs_prof.drop(val)
prs_CG = prs_CG.drop(prs_CG.index[val])
pheno_CG = [y-1 for y in pheno_CG]
prs_CG = prs_CG.fillna(0)
all_matrix_CG = prs_CG

# fit a random forest and get shap values
clf = RandomForestClassifier(max_depth=10, random_state=10)
clf.fit(all_matrix_CG, pheno_CG)
explainer = shap.Explainer(clf)
shap_values_CG = explainer(all_matrix_CG)
# how did it predict?
pred_CG = clf.predict(all_matrix_CG)
accuracy_score(pred_CG,pheno_CG)

In [ ]:
# which snps contribute most to prediction?
shap.summary_plot(shap_values_CG[:,:,1], all_matrix_CG, plot_type='bar')

In [ ]:
# create umap using the shap values
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    s_2d_CG = UMAP(n_neighbors=100, min_dist=0.0, random_state=10).fit_transform(shap_values_CG.values[:, :, 1])

# plot umap projections
plt.figure(figsize=(8, 6))
with warnings.catch_warnings():
    warnings.simplefilter("ignore", UserWarning)
    scatter_CG_neuro = plt.scatter(s_2d_CG[:,0][np.array(pheno_CG)==0], s_2d_CG[:,1][np.array(pheno_CG)==0], label='Neurotypical', c='tab:blue', s=10, alpha=0.5)
    scatter_CG_autistic = plt.scatter(s_2d_CG[:,0][np.array(pheno_CG)==1], s_2d_CG[:,1][np.array(pheno_CG)==1], label='Autistic', c='tab:red', cmap='coolwarm', s=10, alpha=0.5)
plt.legend()
plt.title('UMAP Projection of SHAP Values')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.show()

In [ ]:
# fit kmeans clustering with 6 clusters
kmeans = KMeans(n_clusters=6, random_state=10)
labels_CG = kmeans.fit_predict(s_2d_CG)
colours = ['#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00']

plt.figure(figsize=(8, 6))
for cluster in range(6):
    plt.scatter(
        s_2d_CG[labels_CG == cluster, 0],   # cluster x coords
        s_2d_CG[labels_CG == cluster, 1],   # cluster y coords
        color=colours[cluster],
        label=f'Cluster {cluster}',
        s=10,
        alpha=0.5
    )

# Add legend/labels
plt.legend(title="Clusters")
plt.title('UMAP Projection of SHAP Values')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.savefig('UMAP_clusters.png')
plt.show()

In [ ]:
# what is the breakdown of each cluster?
distribution = pd.DataFrame([labels_CG,pheno_CG,sex_CG, prs_prof['SCORE'], prs_prof['ZSCORE']]).T # also including the actual prses in this df
distribution = distribution.rename(columns={0:'cluster',1:'phenotype',2:'sex',3:'prs',4:'zscore'})

distribution['group'] = distribution['sex'].astype(str) + "," + distribution['phenotype'].astype(str)
label_map = {
    '1.0,0.0': 'male neurotypical',
    '1.0,1.0': 'male autistic',
    '2.0,0.0': 'female neurotypical',
    '2.0,1.0': 'female autistic'
}
distribution['group'] = distribution['group'].map(label_map)

grouped = distribution.groupby(['cluster', 'group']).size().unstack(fill_value=0)

custom_colors = ['#b35806', '#fdb863', '#542788', '#b2abd2'] 
group_labels = ['male neurotypical', 'male autistic', 'female neurotypical', 'female autistic']

grouped[group_labels].plot(kind='bar', stacked=True, figsize=(10,6), color=custom_colors)

plt.xlabel('Cluster')
plt.ylabel('Count')
plt.title('Sex-Phenotype Combinations by Cluster')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# plot prs z-scores (calculated using plink) for each cluster
sns.boxplot(x='cluster', y='zscore', data=distribution)
plt.title('Autism PRS Distribution by Cluster')
plt.show()

In [ ]:
## What about the distribution of all autistic vs all neurotypical?
plt.figure(figsize=(4,6))
sns.boxplot(x='phenotype', y='zscore', data=distribution)
plt.title('Autism PRS Distribution')
plt.savefig('autism_prs_distribution.png')
plt.show()
_, pval = ttest_ind(distribution[distribution['phenotype']==0]['zscore'], distribution[distribution['phenotype']==1]['zscore'])

In [ ]:
# compare the prses between clusters
tukey = pairwise_tukeyhsd(endog=distribution['zscore'],
                          groups=distribution['cluster'], 
                          alpha=0.05)

print(tukey)

In [ ]:
def proportions_ztest(x2, n2): # making my own vers as package wasn't importing from scipy.stats
    p1 = aut_fems / aut_sum
    p2 = x2 / n2
    p_pool = (aut_fems + x2) / (aut_sum + n2)
    se = np.sqrt(p_pool * (1 - p_pool) * (1/aut_sum + 1/n2))
    z_stat = (p1 - p2) / se
    p_value = 2 * (1 - norm.cdf(abs(z_stat)))
    return(p_value)

In [ ]:
# Checking male to female ratio in 0,3,5. None are statistically significantly different to the overall ratio
aut_fems = np.sum(grouped['female autistic'])
aut_sum = aut_fems + np.sum(grouped['male autistic'])

aut_fems0 = grouped.iloc[0]['female autistic']
aut_sum0 = grouped.iloc[0]['female autistic'] + grouped.iloc[0]['male autistic']
pval0 = proportions_ztest(aut_fems0, aut_sum0)
print('cluster 0:', aut_fems0/aut_sum0, pval0)

aut_fems3 = grouped.iloc[3]['female autistic']
aut_sum3 = grouped.iloc[3]['female autistic'] + grouped.iloc[3]['male autistic']
pval3 = proportions_ztest(aut_fems3, aut_sum3)
print('cluster 3:', aut_fems0/aut_sum0, pval3)

aut_fems5 = grouped.iloc[5]['female autistic']
aut_sum5 = grouped.iloc[5]['female autistic'] + grouped.iloc[5]['male autistic']
pval5 = proportions_ztest(aut_fems5, aut_sum5)
print('cluster 5:', aut_fems5/aut_sum5, pval5)

Compare each autistic cluster against all neurotypical

In [ ]:
def predict_autistic_cluster(matrix, labels):
    clf_cluster=RandomForestClassifier(random_state=10)
    clf_cluster.fit(matrix, labels)
    cluster_pred = clf_cluster.predict(matrix)
    print(accuracy_score(cluster_pred, labels))
    explainer = shap.TreeExplainer(clf_cluster)
    shap_values_cluster = explainer(matrix).values
    shap_interaction = explainer.shap_interaction_values(matrix)
    return(shap_values_cluster, shap_interaction)

In [ ]:
# ignore clsuters 3 and 5
mask0 = (labels_CG != 3) & (labels_CG != 5)
labels0 = labels_CG[mask0]
matrix0 = all_matrix_CG[mask0]
# setting labels to be 1 and 0 but since I'm setting all the 0=1 and everything else=0 it req diff steps to the other two
labels0[labels0 != 0] = -1
labels0 = labels0+1
shap_values_cluster0, shap_interaction_cluster0 = predict_autistic_cluster(matrix0, labels0)
cluster0 = matrix0[labels0 == 1]
neurotypical = matrix0[labels0 == 0]

# what contributes most to cluster 0?
shap.summary_plot(shap_values_cluster0[:,:,1], matrix0)

In [ ]:
def compute_signif(cluster, neurotypical):
    pval = []
    for col in np.array(cluster.columns):
        contingency_table = pd.DataFrame({'Cluster': cluster[col].value_counts(),'Neurotypical': neurotypical[col].value_counts()}).fillna(0)
        chi2, p, dof, ex = chi2_contingency(contingency_table)
        pval.append(p)

    pval_df = pd.DataFrame({'snp': cluster.columns,'p-value': pval})

    pval_df['adj p-value'] = pval_df['p-value'] * len(cluster0.columns)
    pval_df['significant'] = pval_df['adj p-value'] < 0.05
    print(pval_df[pval_df['significant'] == True])
    return(pval_df)

def plot_distributions(df1, df2, col, aut_clust_name):
    total_autistic = len(df1)
    total_neurotypical = len(df2)
    combined_df = pd.concat([df1[col].value_counts(normalize=True).sort_index().reindex([0, 1, 2]), 
                             df2[col].value_counts(normalize=True).sort_index().reindex([0, 1, 2])], axis=1)
    combined_df.columns = [aut_clust_name, 'Neurotypical']
    
    ax = combined_df.plot(kind='bar', title=f'Distribution in {col}', figsize=(8,6))
    
    # Add count values above the bars
    for i, p in enumerate(ax.patches):
        height = p.get_height()
        if i < 3:  # First three are the aut ones, second three are the neurotypical
            count_value = height * total_autistic
        else:
            count_value = height * total_neurotypical
        ax.text(p.get_x() + p.get_width() / 2., height + 0.01, f'{int(count_value)}', ha='center', va='bottom')

    plt.show()

In [ ]:
# find which snps are significantly diff between cluster 0 and all neurotypicals
pval_df_cluster0 = compute_signif(cluster0, neurotypical)
# plot the distributions (0,1,2) of that snp
plot_distributions(cluster0, neurotypical, 'rs11787216_T', 'Autistic Cluster 0')

In [ ]:
# ignore autistic clusters 0 and 5, focus on 3 vs neurotypical
mask3 = (labels_CG != 0) & (labels_CG != 5)
labels3 = labels_CG[mask3]
matrix3 = all_matrix_CG[mask3]
labels3[labels3 != 3] = 0
labels3[labels3 == 3] = 1
cluster3 = matrix3[labels3 == 1]
shap_values_cluster3, shap_interaction_cluster3 = predict_autistic_cluster(matrix3, labels3)

#which contributes most to cluster 3?
shap.summary_plot(shap_values_cluster3[:,:,1], matrix3)

In [ ]:
# find which snps are sig diff between cluster 3 and all neurotypicals
pval_df_cluster3 = compute_signif(cluster3, neurotypical)
# plot the distributions of that snp
plot_distributions(cluster3, neurotypical, 'rs112635299_T', 'Autistic Cluster 3')

In [ ]:
# ignore autistic clusters 0 and 3, focus on 5 vs neurotypical
mask5 = (labels_CG != 0) & (labels_CG != 3)
labels5 = labels_CG[mask5]
matrix5 = all_matrix_CG[mask5]
labels5[labels5 != 5] = 0
labels5[labels5 == 5] = 1
shap_values_cluster5, shap_interaction_cluster5 = predict_autistic_cluster(matrix5, labels5)
cluster5 = matrix5[labels5 == 1]

# which contributes most to cluster 5?
shap.summary_plot(shap_values_cluster5[:,:,1], matrix5)

In [ ]:
# find which snps are sig diff between cluster5 and all neurotypical
pval_df_cluster5 = compute_signif(cluster5, neurotypical)
# plot the distribution of that snp
plot_distributions(cluster5, neurotypical, 'rs11787216_T', 'Autistic Cluster 5')